# MVD 8. cvičení

## 1. část - Vytvoření dat

Použijte stejný kód z minulého cvičení pro vytvoření dat. Navíc vytvořte jeden větší dataset, ve kterém sjednotíte výstupy různých funkcí (např. make_blobs + make_circles). 

In [268]:
import numpy as np
from sklearn.datasets import make_blobs, make_moons, make_circles
import pandas as pd

In [269]:

## Small dataset
x_s, y_s = make_blobs(n_samples=100, centers=2, n_features=2,center_box= (-1,1))
df1 = pd.DataFrame({'x': x_s[:, 0], 'y': x_s[:, 1], "label": y_s, "cluster": ["TODO"] * 100})
## big dataset
x1, y1 = make_blobs(n_samples=100, centers=2, n_features=2,center_box= (-1,1))
x2, y2 = make_circles(n_samples=(100,100), noise=0.3,factor=0.2)
y = np.concatenate((y1,y2))
x = np.concatenate((x1,x2), axis=0)
df2 = pd.DataFrame({'x': x[:, 0], 'y': x[:, 1], "label": y, "cluster": ["TODO"] * 300})

x2, y2 = make_blobs(n_samples=200, centers=4, n_features=2,center_box= (-1,1))
df3 = pd.DataFrame({'x': x2[:,0], 'y': x2[:,1],"label":y2,"kmeans":["TODO"]*200})

x3, y3 = make_moons(n_samples=(100,100), noise=0.075)
df4 = pd.DataFrame({'x': x3[:,0], 'y': x3[:,1],"label":y3,"kmeans":["TODO"]*200})

x4, y4 = make_circles(n_samples=(100,100), noise=0.075,factor=0.2)
df5 = pd.DataFrame({'x': x4[:,0], 'y': x4[:,1],"label":y4,"kmeans":["TODO"]*200})

## 2. část - Implementace DBSCAN algoritmu
Dle přednášky implementujte DBSCAN algoritmus.

In [270]:
def close_points(point:pd.DataFrame,df:pd.DataFrame,eps:float) -> pd.DataFrame:
    x, y = point['x'], point['y']
    dist = abs(df['x']-x) + abs(df['y']-y)
    cp = df[dist < eps]
    return cp

def my_dbscan(df:pd.DataFrame,eps:float,min_pts:int):
    ## init method
    df['cluster'] = 'TODO'
    unvisited = list(df.index)
    cluster_id = 0
    cluster_classified = set()
    while len(unvisited) != 0:
        index = unvisited.pop() # select index
        item = df.loc[[index][0]] # get info of selected points
        if item['cluster'] == 'TODO': # if point is not in cluster
            cp = close_points(item,df,eps) # get closed points
            # print(cp)
            # print(df)
            if len(cp) >= min_pts:
                df.loc[index,"cluster"] = cluster_id # make index core point
                cl_points = set(cp.index) ## points that will make cluster
                while len(cl_points) != 0: # until there are points that can make a cluster
                    cp_index = cl_points.pop()
                    cp_item = df.loc[[cp_index][0]]
                    if df.loc[cp_index,'cluster'] == 'TODO':  # if its not already classfied
                        more_points = close_points(cp_item,df,eps)
                        if len(more_points) >=  min_pts:
                            df.loc[cp_index,"cluster"] = cluster_id
                            #print(df.loc[index,"cluster"])
                            cl_points.update(set(more_points.index) - cluster_classified)
                            cluster_classified.update([cp_index])
        if len(cluster_classified) != 0:
            cluster_id += 1
            cluster_classified = set()

## 3. část - Vyhodnocení 
Aplikujte Váš DBSCAN na vytvořené datasety. Experimentálně najděte parametry ```Eps``` a ```MinPts```.

## 4. část - Porovnání s výstupem z knihovny

Porovnejte výstup s výstupem z knihovny sklearn. Dokumentaci naleznete [zde](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).

In [271]:
from sklearn.cluster import DBSCAN
import plotly.express as px

In [272]:
for df_x in [df1,df3,df4,df5,df2]:
    df = df_x

    clustering = DBSCAN(eps=0.5, min_samples=5).fit(df[["x","y"]])

    fig = px.scatter(x=df["x"],y=df["y"], color=[str(x)  for x in df["label"]])
    fig.show()

    fig = px.scatter(x=df["x"],y=df["y"], color=[str(x)  for x in clustering.labels_])
    fig.show()

    my_dbscan(df,0.5,5)
    fig = px.scatter(x=df["x"],y=df["y"], color=[str(x)  for x in df['cluster']])
    fig.show()